In [156]:
from jpnotebooks.other.wql import *
import collections
import types
import datetime
import itertools
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
MIDNA = 'Mîdna'
SERVER = 'Turalyon'
OWNER = "nintendobratkat"
GUILD = "Synthesis"
ZONE = "Tomb of Sargeras"
MYTHIC = 5
api_key = "192b298b50ad53497d9f2b4ce7910af2"

try:
    zraw
except NameError:
    zraw = Api(api_key).zones._raw
    
def connect(api_key=api_key):
    global api
    api = Api(api_key)
    api.zones_ = Zones(api, zraw)
    return api

def pd(o):  # printdir
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o,a,"<Error>")
            if not isinstance(v, (types.FunctionType, types.MethodType)):
                print(a, "::", v)

# TODO: move this to wql.py once I'm confident it works properly


def _clstrfk(events, l):
    try:
        _clstrfk2(events, l)
    except ValueError:
        for c in l:
            for e in c:
                print(e.timestamp, actor(f, e))
            print()
    return l
    
def _clstrfk2(events, l):
    # debug sanity check
    seen = set()
    evd = dict.fromkeys(events, False)
    for c in l:
        for e in c:
            if evd[e] == True:
                raise ValueError("Dulicate event!")
            evd[e] = True
            
def cluster2(events, before=100, after=100):
    l = set()
    for e in events:
        l.add(tuple(events_near(events, e, before, after)))
    _clstrfk(events, l)
    return sorted(map(list,l), key=lambda c: c[0].timestamp)

def cluster(events, before=100, after=100):
    l = cluster2(events, before, after)
    l = [(c[0].timestamp, c) for c in l]
    return sorted(l, key=lambda t: t[0])
   
def hms2(t):
    t = t / 1000
    h = t // 3600
    m = (t-3600*h) // 60
    s = (t-3600*h-60*m)
    ms = (s - int(s))*1000
    return "%.2d:%.2d.%.3d" %(m,s, ms)
                
connect()

In [226]:
def deaths_near_bad(f, *bad, before=100, after=10000, cutoff=8):
    fs_deaths = 'type="death"'
    deaths = [e for e in f.events(fs_deaths, cutoff=cutoff) if e.targetID and f.actor(e.targetID).is_player()]
    fs_bads = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    bads = f.events(fs_bads, cutoff=cutoff)
    fails = set()
    for d in deaths:
        t = tuple(events_near(bads, d, before, after)), d
        fails.add(t)
    return fails

def deaths_near_bad2(f, *bad, before=100, after=10000, cutoff=8):
    fails = deaths_near_bad(f, *bad, before=before, after=after, cutoff=cutoff)
    fails2 = set()
    deaths = set()
    for ev, d in fails:
        fails2.update(ev)
        deaths.add(d)
    key = lambda e: e.timestamp
    return sorted(fails2, key=key), sorted(deaths, key=key)

In [227]:
print(f.name, f.id)
fails, deaths = deaths_near_bad2(f, "Rain of Brimstone")
for fail in fails:
    print(actor(f, fail))

Goroth 2
Lilboov
Sonohana
Earthgroans
Kôrboov
Usmcwind
Abradin
Dookú
Madduck
Sìeara
Goblinguide
Grouper


# Goroth

In [232]:
reports = []
bosses = {
    "Goroth": [],
    "Demonic Inquisition": [],
    "Harjatan": [],
    "Sisters of the Moon": [],
    "The Desolate Host": []
}
start = datetime.datetime.now() - datetime.timedelta(weeks=1)
for r in api.reports(GUILD, SERVER, "US").find(owner=OWNER, zone=ZONE, func=lambda r: r.start > start):
    fights = r.fights()
    for boss in bosses:
        fl = fights.find(difficulty=MYTHIC, name=boss, kill=True)
        bosses[boss].extend(fl)
        
for v in bosses.values():
    v.sort(key=lambda f: f.id)
bosses

{'Demonic Inquisition': [<jpnotebooks.other.wql.Fight at 0x1abe3cae278>],
 'Goroth': [<jpnotebooks.other.wql.Fight at 0x1abe3cae240>],
 'Harjatan': [<jpnotebooks.other.wql.Fight at 0x1abe3cae208>],
 'Sisters of the Moon': [<jpnotebooks.other.wql.Fight at 0x1abe3c44828>],
 'The Desolate Host': []}

In [255]:
def actor(f, e):
    return f.actor(e.targetID).name

def spike(f, cutoff=8):
    fs = 'type="damage" AND ability.name="Infernal Spike"'
    events = f.events(fs, cutoff=cutoff)
    for e in events:
        print(actor(f,e))
        
def ft(f,e):
    return (e.timestamp-f.start_time)

def _bms_output1(f, c, good, bad, dl):
        # finally, output
        width = 15
        fmt = "%%%ss | %%%ss | %%%ss" % (width, width, width)
        fmt = "%-15s | %-15s | %-15s"
        out = list(itertools.zip_longest(good, bad, dl, fillvalue=""))
        print(fmt%("Good", "Bad", "Deaths"))
        print("-"*(3*width))
        for row in out:
            print(fmt%row)
        print()
        
pindent = 0
def print2(*args, **kw):
    if pindent > 0:
        print(pindent*"  ", *args, **kw)
    else:
        print(*args, **kw)
        
def _bms_output2(f, c, good, bad, dl):
    print2("Time:", hms2(ft(f,c[0])))
    print2("Good:", ",".join(good))
    print2("Bad:", ",".join(bad))
    print2("Deaths:", ",".join(dl))
    print2()
    
        
def brimstone(f, cutoff=8):
    fs_rain = 'type="damage" AND ability.name="Rain of Brimstone"'
    immunities = "Cloak of Shadows", "Ice Block", "Divine Shield", "Aspect of the Turtle", "Netherwalk"
    fs_immunity = 'type="applybuff" AND (%s)' % " OR ".join('ability.name="%s"'%im for im in immunities)
    fs_deaths = 'type="death"'
    
    rains = f.events(fs_rain, cutoff=cutoff)
    immunities = f.events(fs_immunity, cutoff=cutoff)
    deaths = f.events(fs_deaths, cutoff=cutoff)
    
    players = {fr.name for fr in f.friendlies.players()}
    
    hits = cluster2(rains, 1000, 1000)
    for c in hits:
        im = events_near(immunities, c[0], 8000, 0)
        c.extend(im)
        
    for c in hits:
        # start slightly before to account for combat log timestamps...
        rd = events_near(deaths, c[0], 500, 5000)
        good = sorted(actor(f,e) for e in c)
        bad = sorted(players - set(good))
        dl = sorted(actor(f,e) for e in rd if e.targetID and f.actor(e.targetID).is_player())
        #_bms_output1(f, c, good, bad, dl)
        _bms_output2(f, c, good, bad, dl)

def infernal_spikes(f, cutoff=8):
    fs = 'type="damage" AND ability.name="Infernal Spike"'
    el = f.events(fs, cutoff=cutoff)
    for e in el:
        print2(actor(f,e))
        
        
def spy(f, name, func, *args, **kw):
    global pindent
    pindent=1
    print2(name)
    pindent=2
    func(f, *args, **kw)
    pindent=0
    
    
api.verbose = False
for f in bosses['Goroth']:
    print2("Fight: Goroth", f.id)
    print2()
    spy(f, "Rain of Brimstone", brimstone)
    spy(f, "Infernal Spike", infernal_spikes)

Fight: Goroth 2

   Rain of Brimstone
     Time: 00:20.283
     Good: Decenary,Dookú,Earthgroans,Goblinguide,Kôrboov,Leafý,Lilboov,Madduck,Shadx,Shamoura,Sìeara,Usmcwind
     Bad: Abradin,Damndruids,Grasping,Grouper,Lilshadooww,Midnaught,Sonohana,Xaere
     Deaths: 
    
     Time: 01:21.144
     Good: Abradin,Damndruids,Decenary,Dookú,Earthgroans,Goblinguide,Grouper,Leafý,Madduck,Shadx,Shamoura,Sonohana,Sìeara,Usmcwind
     Bad: Grasping,Kôrboov,Lilboov,Lilshadooww,Midnaught,Xaere
     Deaths: 
    
     Time: 02:22.051
     Good: Dookú,Earthgroans,Goblinguide,Grasping,Grouper,Kôrboov,Leafý,Lilshadooww,Madduck,Shadx,Shamoura,Sonohana,Usmcwind
     Bad: Abradin,Damndruids,Decenary,Lilboov,Midnaught,Sìeara,Xaere
     Deaths: 
    
     Time: 03:22.247
     Good: Abradin,Decenary,Goblinguide,Grouper,Kôrboov,Leafý,Lilboov,Sonohana,Sìeara,Usmcwind
     Bad: Damndruids,Dookú,Earthgroans,Grasping,Lilshadooww,Madduck,Midnaught,Shadx,Shamoura,Xaere
     Deaths: Decenary,Leafý
    
     Time: 0

# Sisters of the Moon Weekend Prog Kill

In [ ]:
def scan_for_gold_rule_glaive_storm(f, **other):
    start = f.start_time
    fs = '(type="damage" AND ability.name="Glaive Storm") OR type="death"'
    gs = []
    deaths = []
    while True:
        events = f.api.events(f.report_id, start, f.end_time, filter=fs, **other)
        start = events.nextPageTimestamp
        for e in events.events:
            id = e.targetID
            if id and f.actor(id).is_player():
                if e.type == 'death':
                    deaths.append(e)
                else:
                    gs.append(e)
        if start is None or start > f.end_time:
            break
    return gs, deaths

def treasure(f, gs, deaths):
    # O(n^2) scan ffffffff
    owed = []
    for g in gs:
        tsg = g.timestamp
        for d in deaths:
            tsd = d.timestamp
            if 0 <= tsd - tsg <= 5000:
                owed.append((f.actor(g.targetID)))
                break
    return owed

In [ ]:
import collections

book = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
connect(api_key)
api.verbose = False
reports = api.reports("Synthesis", "Turalyon").find(owner='nintendobratkat', zone="Tomb of Sargeras")
for r in reports:
    fights = r.fights()
    for f in fights.find(name="Sisters of the Moon", difficulty=MYTHIC):
        for cutoff in range(1, 10):
            gs, deaths = scan_for_gold_rule_glaive_storm(f, cutoff=cutoff)
            owed = treasure(f, gs, deaths)
            for player in owed:
                book[player.name][cutoff] += 1
                

In [ ]:
s = "%12s:%s"%("Death Cutoff", " ".join("%4d" % c for c in range(1, 10)))

out = []
def print_(s):
    print(s)
    out.append(s)
print_("Number of people hit by glaive storm <5 sec before a death:\n")
print_(s)
print_("-"*len(s))
for name in sorted(book, key=lambda p: book[p][max(book[p])], reverse=True):
    cutoffs = book[name]
    s = "%12s: %s"%(name, ", ".join("%3d" % cutoffs[c] for c in range(1, 10)))
    print_(s)        

# Maiden Week 1 Fails

In [4]:
def iter_events(f, fs, **other):
    start = f.start_time
    while True:
        events = f.api.events(f.report_id, start, f.end_time, filter=fs, **other)
        start = events.nextPageTimestamp
        for e in events.events:
            yield e
        if start is None or start > f.end_time:
            break

def ok_us(m, us):
    for u in us:
        if m < u and (u-m) < 2:
            return True
    return False

            
def bad_unstable(f):
    fs = "type='applydebuff' AND ability.name='Unstable Soul'"
    us = list(iter_events(f, fs, cutoff=2))
    fs = 'type="cast" AND ability.name="Mass Instability"'
    mi = list(iter_events(f, fs, cutoff=2))
    u_bad = set()
    u_good = set()
    for m in mi:
        u_ok = set()
        for u in us:
            if u.timestamp>m.timestamp and (u.timestamp-m.timestamp) < 3000:
                u_ok.add(u)
        if len(u_ok) > 3:
            u_bad.update(u_ok)
        else:
            u_good.update(u_ok)
    #u_bad.update(set(us) - u_good)
    u_bad = set(us) - u_good
    u_bad = sorted(u_bad, key=lambda e: e.timestamp)
    return u_bad, us
            

SyntaxError: invalid syntax (<ipython-input-4-17f3186fcd5b>, line 20)

In [9]:
def ts2hms(t):
    t = t / 1000
    h = t // 3600
    m = (t-3600*h) // 60
    s = (t-3600*h-60*m)
    ms = (s - int(s))*1000
    return "%d:%.2d:%.2d.%.3d" %(h,m,s, ms)

def hms2(t):
    t = t / 1000
    h = t // 3600
    m = (t-3600*h) // 60
    s = (t-3600*h-60*m)
    ms = (s - int(s))*1000
    return "%.2d:%.2d.%.3d" %(m,s, ms)

In [ ]:
import collections
connect()
reports = api.reports("Synthesis", "Turalyon").find(id="Qx6Vc7wvZ1LBJTnk")
report = reports.index(0)
players_bad = collections.defaultdict(list)
players_all = collections.defaultdict(list)
api.verbose = False
for f in report.fights():
    bad, us = bad_unstable(f)
    #print(f, ", ".join(f.actor(e.targetID).name for e in bad))
    for e in bad:
        players_bad[f.actor(e.targetID).name].append((f, e))
    for e in us:
        players_all[f.actor(e.targetID).name].append((f, e))

In [ ]:
fc = [(player, len(fails)) for player, fails in players_bad.items()]
for player, fails in sorted(fc, key=lambda t: t[1], reverse=True):
    print("%-15s"%player, fails)

In [ ]:
def bad_unstable2(f):
    fs = "type='applydebuff' AND ability.name='Unstable Soul'"
    us = list(iter_events(f, fs, cutoff=2))
    fs = 'type="cast" AND ability.name="Mass Instability"'
    mi = list(iter_events(f, fs, cutoff=2))
    u_bad = set()
    u_good = set()
    for m in mi:
        print("MI:", ts2hms(m.timestamp-f.start_time))
        u_ok = set()
        for u in us:
            if u.timestamp >= m.timestamp:
                print("US:", ts2hms(u.timestamp-f.start_time))
            if u.timestamp>m.timestamp and (u.timestamp-m.timestamp) < 3000:
                u_ok.add(u)
        if len(u_ok) > 3: 
            u_bad.update(u_ok)
        else:
            u_good.update(u_ok)
    u_bad.update(set(us) - u_good)
    u_bad = sorted(u_bad, key=lambda e: e.timestamp)
    return u_bad, us
for f in report.fights():
    print(f)
    bad_unstable2(f)


In [ ]:
def bad_unstable3(f):
    # core debuffs
    us = list(iter_events(f, "type='applydebuff' AND ability.name='Unstable Soul'", cutoff=2))
    lif = list(iter_events(f, "type='applydebuff' AND ability.name='Light Infusion'", cutoff=2))
    fif = list(iter_events(f, "type='applydebuff' AND ability.name='Fel Infusion'", cutoff=2))
    inf_both = lif+fif
    
    # sources
    mi = list(iter_events(f, 'type="cast" AND ability.name="Mass Instability"', cutoff=2))
    swirlies = list(iter_events(f, 'type="damage" AND (ability.name="Light Echoes" OR ability.name="Fel Echoes")'))
    
    u_bad = set()
    u_good = set()
    for m in mi:
        u_ok = set()
        for u in us:
            if u.timestamp>m.timestamp and (u.timestamp-m.timestamp) < 3000:
                u_ok.add(u)
        if len(u_ok) > 3: 
            u_bad.update(u_ok)
        else:
            u_good.update(u_ok)
    u_bad.update(set(us) - u_good)
    u_bad = sorted(u_bad, key=lambda e: e.timestamp)
    
    inf = None
    inf_ts = 0
    for u in u_bad:
        player = f.actor(u.targetID)
        ts = u.timestamp
        for s in swirlies:
            for e in inf_both:
                if e.timestamp > s.timestamp:
                    break
                if e.targetID == u.targetID and e.timestamp > inf_ts:
                    inf = e.ability.name
                    inf_ts = e.timestamp
            if s.targetID == u.targetID and abs(s.timestamp-u.timestamp) < 500 and s.ability.name != inf:
                print(f, "%s: Got from Swirly %s!"%(f.actor(s.targetID).name, s.ability.name))
        
    
    return u_bad, us
for f in report.fights():
    #print(f)
    bad, us = bad_unstable3(f)

In [ ]:
def ets(f,e):
    return(ts2hms(e.timestamp-f.start_time))

def bad_unstable4(f):
    player_status = {}
    mass_infusion = 0
    def actor(e):
        return f.actor(e.targetID).name
    def tss(f,e):
        return hms2(e.timestamp-f.start_time)
    debuffs = "Unstable Soul", "Light Infusion", "Fel Infusion"
    fdb = "type='applydebuff' AND (%s)" % " OR ".join("ability.name='%s'" % a for a in debuffs)
    mi = 'type="cast" AND ability.name="Mass Instability"'
    swirlies = 'type="damage" AND (ability.name="Light Echoes" OR ability.name="Fel Echoes")'
    fs = " OR ".join("(%s)"%f for f in (fdb, mi, swirlies))
    for e in f.events(fs, cutoff=3):
        if e.type == "applydebuff":
            print("%-15s"%actor(e), "%-12s"% e.type, "%-10s"%tss(f,e), e.ability.name)
            if ability.name == "Fel Infusion" or ability.name == "Light Infusion":
                player_status[actor(e)]['infusion'] = e.ability.name
            else:  # unstable soul
                player_status[actor(e)]['unstable'] = e.timestamp
        elif e.type == "cast":
            print("%-15s"%"", "%-12s"%e.type, "%-10s"%tss(f,e), e.ability.name)
            mass_infusion = e.timestamp
        elif e.type == "damage":
            print("%-15s"%actor(e), "%-12s"% e.type, "%-10s"%tss(f,e), e.ability.name)
            player_status[actor(e)]['swirly'] = e.timestamp
        else:
            print("UH OH:", e.type)
for f in report.fights():
    print(f)
    bad_unstable4(f)
    break

## Bad Unstable 5 (The good one!)

In [10]:
def events_near(events, ev, before, after):
    rv = []
    b = ev.timestamp - before
    a = ev.timestamp + after
    for e in events:
        if e.timestamp >= b:
            if e.timestamp <= a:
                rv.append(e)
            else:
                break
    return rv
evnr = events_near

def clusterify(events, before=100, after=100):
    l = set()
    for e in events:
        l.add(tuple(events_near(events, e, before, after)))
    l = [(c[0].timestamp, c) for c in l]
    
    # debug sanity check
    for c in l:
        saw = False
        for e in events:
            if e in c:
                if saw == True:
                    raise ValueError("event in multiple clusters")
                saw = True
    return sorted(l, key=lambda t: t[0])

all_bad = collections.defaultdict(list)

def bad_unstable5(f):
    infusions = {}
    mass_instability = 0
    us = {fr.name: False for fr in f.friendlies}
    
    def actor(e):
        return f.actor(e.targetID).name
    def tss(f,e):
        return hms2(e.timestamp-f.start_time)
    
    dbl = "Light Infusion", "Fel Infusion"
    infusion = "type='applydebuff' AND (%s)" % " OR ".join("ability.name='%s'" % a for a in dbl)
    mi = 'type="cast" AND ability.name="Mass Instability"'
    swirlies = 'type="damage" AND (ability.name="Light Echoes" OR ability.name="Fel Echoes")'
    usdmg = 'type="damage" AND ability.name="Unstable Soul"'
    us_fade = 'type="removedebuff" AND ability.name="Unstable Soul"'
    us_apply = 'type="applydebuff" AND ability.name="Unstable Soul"'
    #deaths = 'type="death" AND target.type="player"'
    
    events = []
    
    infusion_events = f.events(infusion, cutoff=3)
    mi_events       = f.events(mi, cutoff=3)
    swirly_events   = f.events(swirlies, cutoff=3)
    usdmg_events    = f.events(usdmg, cutoff=3)
    us_fade_events  = f.events(us_fade, cutoff=3)
    us_apply_events = f.events(us_apply, cutoff=3)
    
    us_clusters = clusterify(us_apply_events)
    print("\nClustered Unstable Soul Events:")
    for ts, c in us_clusters:
        s = "%-8s"%"<good?>" if ((len(c) <= 3) and (len(events_near(mi_events, c[0], 2500, -1000)))) else ("%-8s"%"<bad>")
        print(hms2(ts-f.start_time), s, ", ".join(map(actor, c)))
    print()
    
    for el in (infusion_events, mi_events, swirly_events, usdmg_events, us_fade_events, us_apply_events):
        events.extend(el)    
    events.sort(key=lambda e: e.timestamp)
    
    for e in events:
        if e.type == "applydebuff":
            if e.ability.name == "Unstable Soul":
                src = ""
                if any(e.targetID == ev.targetID for ev in events_near(swirly_events, e, 200, 200)):  # combat log can record events *slightly* out of order...
                    src = "Swirly"
                elif events_near(mi_events, e, 3000, 0):
                    near_mi = events_near(us_apply_events, e, 3000, 100)
                    if len(near_mi) <= 3:  # every now and then this can be < 3, somehow. not sure. 
                        src = "Mass Instability"
                    else:
                        # scan to figure out which of are outliers
                        bad = []
                        for ev in near_mi:
                            if len(events_near(near_mi, ev, 50, 50)) != 3:
                                bad.append(ev)
                        src = "baddies: " + ", ".join(f.actor(ev.targetID).name for ev in bad)
                        for ev in bad:
                            all_bad[actor(ev)].append(ev)
                else:                    
                    src = "bad"
                    all_bad[actor(e)].append(e)
                print("%-15s"%actor(e), "%-12s"% e.type, "%-10s"%tss(f,e), "%-15s"%e.ability.name, src)
                if us[actor(e)] == True:
                    print("Detonate!")
                us[actor(e)] = True
            else:
                infusions[actor(e)] = e.ability.name
        elif e.type == "removedebuff":
            if e.ability.name == "Unstable Soul":
                #print("%-15s"%actor(e), "%-12s"% e.type, "%-10s"%tss(f,e), e.ability.name)
                us[actor(e)] = False
        elif e.type == "cast":
            print("%-15s"%"<Boss>", "%-12s"%e.type, "%-10s"%tss(f,e), e.ability.name)
            mass_instability = e.timestamp
        elif e.type == "damage":
            if e.ability.name != "Unstable Soul":
                print("%-15s"%actor(e), "%-12s"% e.type, "%-10s"%tss(f,e), "%-15s"%e.ability.name) # , "%-10s"%("<%s>"%e.ability.guid))
            if e.ability.guid == 235138 and e.amount > 3000000:
                print("%-15s"%f.actor(e.sourceID).name, "%-12s"% "DETONATE", "%-10s"%tss(f,e), "%-15s"%e.ability.name, "%-15s"%actor(e)) # , "%-10s"%("<%s>"%e.ability.guid))
                #print(e.amount, e.hitPoints, e.maxHitPoints, e.absorbed, e.overkill)
                if e.hitPoints == 0:
                    break
        elif e.type == "death":
            pass
            #print("%-15s"%actor(e), "%-12s"% e.type, "%-10s"%tss(f,e))
        else:
            print("UH OH:", e.type)

connect()
reports = api.reports("Synthesis", "Turalyon").find(id="Qx6Vc7wvZ1LBJTnk")
report = reports.index(0)
            
for i, f in enumerate(report.fights()):
    print(f)
    bad_unstable5(f)
    print()
    
fc = [(player, len(fails)) for player, fails in all_bad.items()]
for player, fails in sorted(fc, key=lambda t: t[1], reverse=True):
    print("%-15s"%player, fails)

<Fight: 1, 'Maiden of Vigilance'>

Clustered Unstable Soul Events:
00:12.544 <bad>    Dichotomous, Negative, Mesmerize, Nangz, Tyler
00:18.053 <bad>    Horticulture

Dichotomous     applydebuff  00:12.544  Unstable Soul   bad
Negative        applydebuff  00:12.544  Unstable Soul   bad
Mesmerize       applydebuff  00:12.544  Unstable Soul   bad
Nangz           applydebuff  00:12.544  Unstable Soul   bad
Tyler           applydebuff  00:12.544  Unstable Soul   bad
Horticulture    applydebuff  00:18.053  Unstable Soul   Swirly
Horticulture    damage       00:18.057  Light Echoes   
Nangz           DETONATE     00:18.608  Unstable Soul   Kuraga         
Nangz           DETONATE     00:18.640  Unstable Soul   Vralk          

<Fight: 2, 'Maiden of Vigilance'>
Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=171009&filter=type%3D%27applydebuff%27+AND+%28ability.name%3D%27Light+Infusion%27+OR+ability.name%3D%27Fel+Infusion%27%29&end=206226&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'
Re

Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=910035&filter=type%3D%22damage%22+AND+ability.name%3D%22Unstable+Soul%22&end=945823&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'
Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=910035&filter=type%3D%22removedebuff%22+AND+ability.name%3D%22Unstable+Soul%22&end=945823&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'
Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=910035&filter=type%3D%22applydebuff%22+AND+ability.name%3D%22Unstable+Soul%22&end=945823&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'

Clustered Unstable Soul Events:
00:16.696 <bad>    Dichotomous
00:26.050 <good?>  Dichotomous, Vralk, Smuft
00:28.614 <bad>    Rhogax
00:29.280 <bad>    Novasan

Dichotomous     applydebuff  00:16.696  Unstable Soul   bad
<Boss>          cast         00:24.044  Mass Instability
Dichotomous     applydebuff  00:26.050  Unstable Soul   Mass Instability
Vralk           applydebuff  00:26.050  Unstable Soul   Mass Instability
Smuft   

Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=1750538&filter=type%3D%22cast%22+AND+ability.name%3D%22Mass+Instability%22&end=1805702&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'
Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=1750538&filter=type%3D%22damage%22+AND+%28ability.name%3D%22Light+Echoes%22+OR+ability.name%3D%22Fel+Echoes%22%29&end=1805702&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'
Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=1750538&filter=type%3D%22damage%22+AND+ability.name%3D%22Unstable+Soul%22&end=1805702&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'
Requesting '/report/events/Qx6Vc7wvZ1LBJTnk?start=1750538&filter=type%3D%22removedebuff%22+AND+ability.name%3D%22Unstable+Soul%22&end=1805702&api_key=192b298b50ad53497d9f2b4ce7910af2&cutoff=3'


KeyboardInterrupt: 

In [37]:
f.friendlies.friendlies
fs = [fr for fr in f.friendlies if fr.is_player()]
fs.sort(key=lambda fr: fr.id)
report.fights()._raw['friendlies']

[{'fights': [{'id': 1},
   {'id': 2},
   {'id': 3},
   {'id': 4},
   {'id': 5},
   {'id': 6},
   {'id': 7},
   {'id': 8},
   {'id': 9},
   {'id': 10},
   {'id': 11},
   {'id': 12},
   {'id': 13},
   {'id': 14},
   {'id': 15},
   {'id': 16},
   {'id': 17},
   {'id': 18},
   {'id': 19},
   {'id': 20},
   {'id': 21},
   {'id': 22},
   {'id': 23},
   {'id': 24},
   {'id': 25},
   {'id': 26},
   {'id': 27},
   {'id': 28},
   {'id': 29},
   {'id': 30},
   {'id': 31},
   {'id': 32},
   {'id': 33},
   {'id': 34},
   {'id': 35},
   {'id': 36},
   {'id': 37},
   {'id': 38},
   {'id': 39},
   {'id': 40},
   {'id': 41},
   {'id': 42},
   {'id': 43},
   {'id': 44},
   {'id': 45},
   {'id': 46},
   {'id': 47},
   {'id': 48},
   {'id': 49},
   {'id': 50},
   {'id': 51},
   {'id': 52},
   {'id': 53}],
  'guid': 148798769,
  'id': 10,
  'name': 'Damnatíon',
  'type': 'Warlock'},
 {'fights': [{'id': 1},
   {'id': 2},
   {'id': 3},
   {'id': 4},
   {'id': 5},
   {'id': 6},
   {'id': 7},
   {'id': 8},
   

In [28]:
r = f.api.events(f.report_id, f.start_time, f.end_time)
import json
print(json.dumps(r['events'][1], indent=4))

{
    "pvpTalents": [
        {
            "icon": "ability_pvp_gladiatormedallion.jpg",
            "id": 208683
        },
        {
            "icon": "spell_arcane_mindmastery.jpg",
            "id": 213549
        },
        {
            "icon": "spell_shadow_dispersion.jpg",
            "id": 199131
        },
        {
            "icon": "ability_priest_shadowyapparition.jpg",
            "id": 228630
        },
        {
            "icon": "ability_priest_psychiclink.jpg",
            "id": 199484
        },
        {
            "icon": "spell_priest_psyfiend.jpg",
            "id": 211522
        }
    ],
    "agility": 7501,
    "talents": [
        {
            "icon": "spell_shadow_mindtwisting.jpg",
            "id": 109142
        },
        {
            "icon": "spell_holy_symbolofhope.jpg",
            "id": 64129
        },
        {
            "icon": "spell_shadow_mindbomb.jpg",
            "id": 205369
        },
        {
            "icon": "spell_shadow_

In [29]:
r['events'][1].keys()

dict_keys(['pvpTalents', 'agility', 'talents', 'critMelee', 'hasteRanged', 'leech', 'dodge', 'block', 'sourceID', 'specID', 'hasteMelee', 'versatilityDamageDone', 'critSpell', 'gear', 'versatilityDamageReduction', 'stamina', 'avoidance', 'type', 'armor', 'timestamp', 'speed', 'artifact', 'strength', 'mastery', 'hasteSpell', 'critRanged', 'auras', 'parry', 'versatilityHealingDone', 'intellect'])

In [30]:
r['events'][1]['sourceID']

22

In [31]:
f.actor(22)

<Friendly: 22, 'Kuraga', 'Priest'>

In [33]:
for e in r['events']:
    if e['type'] == 'combatantinfo':
        print(f.actor(e.sourceID))

<Friendly: 22, 'Kuraga', 'Priest'>
<Friendly: 2, 'Vralk', 'Priest'>
<Friendly: 10, 'Damnatíon', 'Warlock'>
<Friendly: 14, 'Novasan', 'Monk'>
<Friendly: 1, 'Dichotomous', 'Shaman'>
<Friendly: 9, 'Mesmerize', 'Rogue'>
<Friendly: 13, 'Rhogax', 'Mage'>
<Friendly: 8, 'Benderic', 'Priest'>
<Friendly: 7, 'Alèsana', 'Druid'>
<Friendly: 6, 'Mîdna', 'Shaman'>
<Friendly: 5, 'Nyteki', 'Priest'>
<Friendly: 11, 'Rikachu', 'Rogue'>
<Friendly: 16, 'Negative', 'DeathKnight'>
<Friendly: 12, 'Expose', 'Hunter'>
<Friendly: 18, 'Tyler', 'Paladin'>
<Friendly: 23, 'Smuft', 'DeathKnight'>
<Friendly: 4, 'Nangz', 'DeathKnight'>
<Friendly: 17, 'Thorrior', 'Warrior'>
<Friendly: 15, 'Horticulture', 'Druid'>
<Friendly: 3, 'Tojura', 'Hunter'>


In [ ]:
fights